In [75]:
import numpy as np
import os
import pandas as pd
import re
import requests
import kagglehub
import sys

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Machine Learning Project

by Andon Gorchov (@thunderman913)

In [32]:
path = kagglehub.dataset_download("tsaustin/us-historical-stock-prices-with-earnings-data")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\andon\.cache\kagglehub\datasets\tsaustin\us-historical-stock-prices-with-earnings-data\versions\7


In [33]:
earnings_path = f"{path}\\stocks_latest\\earnings_latest.csv"
stock_prices_path = f"{path}\\stocks_latest\\stock_prices_latest.csv"

In [34]:
earnings_data = pd.read_csv(earnings_path)
stock_data = pd.read_csv(stock_prices_path)

In [35]:
# Function to get the price X days ago
def get_price_x_days_ago(df, days):
    return df['close'].shift(days)

# Function to get peak price and how many days ago it happened
def get_peak_price_and_days_ago(df):
    df['peak_price'] = df['close'].rolling(window=90, min_periods=1).max()
    peak_day_index = df['close'].rolling(window=90, min_periods=1).apply(lambda x: (len(x) - 1 - x.argmax()), raw=False)
    df['peak_days_ago'] = peak_day_index.fillna(0).astype(int)  # Fill NaN for rows that don't have enough data
    return df

# Apply the operations without grouping by ticker
def process_ticker_data(df):
    # Add columns for prices X days ago
    for days in [1,2,3,4,5, 10, 15, 30, 60, 90]:
        df[f'price_{days}_days_ago'] = df.groupby('ticker')['close'].shift(days)
        df[f'volume_{days}_days_ago'] = df.groupby('ticker')['volume'].shift(days)
    
    # Get peak price and peak days ago
    #df = df.groupby('ticker', group_keys=False).apply(get_peak_price_and_days_ago)
    return df

In [36]:
def tidyStockData():
    file_path = 'stock_data_processed.pkl'

    # Check if the file exists
    if os.path.exists(file_path):
        return pd.read_pickle('stock_data_processed.pkl')

    stock_data_tidy = stock_data

    stock_data_tidy.date = pd.to_datetime(stock_data_tidy.date)
    stock_data_tidy = stock_data_tidy.rename(columns={'symbol': 'ticker'})
    stock_data_tidy = stock_data_tidy.drop(columns=['split_coefficient'])

    stock_data_tidy = stock_data_tidy[(stock_data_tidy['date'] > (earnings_data_tidy['date'].min() - pd.Timedelta(days=100)))
                                    | (stock_data_tidy['date'] < earnings_data_tidy['date'].max() + pd.Timedelta(days=5))]

    # Sort the DataFrame by ticker and date
    stock_data_tidy = stock_data_tidy.sort_values(by=['ticker', 'date'], ascending=[True, True])

    # Process the DataFrame without grouping
    stock_data_tidy = process_ticker_data(stock_data_tidy)

    stock_data_tidy = stock_data_tidy[stock_data_tidy.price_90_days_ago.isna() == False]
    stock_data_tidy.to_pickle('stock_data_processed.pkl')
    
    return stock_data_tidy

In [37]:
stock_data_tidy = tidyStockData()

In [38]:
earnings_data_tidy = earnings_data
earnings_data_tidy.date = pd.to_datetime(earnings_data_tidy.date)

In [39]:
earnings_data_tidy = earnings_data_tidy.rename(columns={'symbol': 'ticker'})

In [40]:
earnings_data_tidy = earnings_data_tidy.drop(earnings_data_tidy[(earnings_data_tidy.eps.isna()) | (earnings_data_tidy.eps_est.isna())].index)
earnings_data_tidy = earnings_data_tidy.drop(columns=['qtr'])

In [41]:
earnings_data_tidy = earnings_data_tidy.rename(columns={'date': 'earnings_date'})

In [42]:
earnings_data_tidy['release_time'] = earnings_data_tidy.groupby('ticker')['release_time'].ffill().bfill()

In [43]:
# Adding earnings data to stock_data_tidy based on the nearest previous date
def add_earnings_data(stock_data, earnings):
    # Merge earnings data and stock data on ticker
    merged_df = pd.merge_asof(
        stock_data.sort_values(by='date'),
        earnings.sort_values(by='earnings_date'),
        by='ticker',
        left_on='date',
        right_on='earnings_date',
        direction='backward'
    )
    
    # Forward fill to ensure non-trading days are adjusted properly
    merged_df = merged_df.sort_values(by=['ticker', 'date'])

    # Merge again to get the next earnings date
    merged_df = pd.merge_asof(
        merged_df.sort_values(by='date'),
        earnings[['ticker', 'earnings_date']].sort_values(by='earnings_date'),
        by='ticker',
        left_on='date',
        right_on='earnings_date',
        direction='forward',
        suffixes=('', '_next'),
        allow_exact_matches=False
    )
    
    return merged_df

# Add earnings data to stock_data_tidy
stock_data_tidy = add_earnings_data(stock_data_tidy, earnings_data_tidy)

In [44]:
earnings_data_tidy[earnings_data_tidy.ticker == 'PAYS']

,ticker,earnings_date,eps_est,eps,release_time
115962,PAYS,2019-05-07,0.020,0.03,pre
115963,PAYS,2019-05-08,0.020,0.03,pre
115964,PAYS,2019-08-06,0.033,0.05,pre
115965,PAYS,2019-11-05,0.040,0.06,post
115966,PAYS,2020-04-03,0.060,0.05,pre
115967,PAYS,2020-08-13,0.035,0.01,post
115968,PAYS,2020-11-17,0.018,-0.14,pre
115969,PAYS,2021-03-25,0.013,0.01,pre
115970,PAYS,2021-05-11,0.013,-0.01,post


In [45]:
stock_data_tidy = stock_data_tidy.drop(stock_data_tidy[(stock_data_tidy.earnings_date.isna()) | (stock_data_tidy.earnings_date_next.isna())].index)

In [54]:
stock_data_tidy['days_till_report'] = (stock_data_tidy['earnings_date_next'] - stock_data_tidy['date']).dt.days
stock_data_tidy['days_since_report'] = (stock_data_tidy['date'] - stock_data_tidy['earnings_date']).dt.days

In [58]:
stock_data_tidy[['ticker','date', 'earnings_date', 'earnings_date_next', 'days_till_report', 'days_since_report']]

,ticker,date,earnings_date,earnings_date_next,days_till_report,days_since_report
0,A,2012-12-26,2012-11-19,2013-02-14,50,37
2,PAR,2012-12-26,2012-11-05,2013-02-14,50,51
3,AIZ,2012-12-26,2012-10-24,2013-02-06,42,63
7,PAYX,2012-12-26,2012-12-19,2013-03-27,91,7
8,PB,2012-12-26,2012-10-24,2013-01-25,30,63
...,...,...,...,...,...,...
11762357,RFIL,2021-06-10,2021-03-11,2021-06-14,4,91
11762404,CODA,2021-06-10,2021-03-17,2021-06-14,4,85
11762995,ADXS,2021-06-11,2020-09-10,2021-06-14,3,274
11764002,RFIL,2021-06-11,2021-03-11,2021-06-14,3,92


We remove data with more than 120 days on both next and previous report, since that is useless data for our case

In [67]:
stock_data_tidy = stock_data_tidy.drop(stock_data_tidy[(stock_data_tidy.days_till_report > 120) & (stock_data_tidy.days_since_report > 120)].index)

In [70]:
stock_data_tidy = stock_data_tidy.drop(columns=['open', 'high', 'low', 'close_adjusted', 'volume'])

In [73]:
stock_data_tidy.columns

Index(['ticker', 'date', 'close', 'peak_price', 'peak_days_ago',
       'price_1_days_ago', 'volume_1_days_ago', 'price_2_days_ago',
       'volume_2_days_ago', 'price_3_days_ago', 'volume_3_days_ago',
       'price_4_days_ago', 'volume_4_days_ago', 'price_5_days_ago',
       'volume_5_days_ago', 'price_10_days_ago', 'volume_10_days_ago',
       'price_15_days_ago', 'volume_15_days_ago', 'price_30_days_ago',
       'volume_30_days_ago', 'price_60_days_ago', 'volume_60_days_ago',
       'price_90_days_ago', 'volume_90_days_ago', 'earnings_date', 'eps_est',
       'eps', 'release_time', 'earnings_date_next', 'days_till_report',
       'days_since_report'],
      dtype='object')